In [1]:
using TensorKit
import TensorKit.×
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
include("./models/Hubbard_Z2SU2.jl")
@show pkgversion(TensorKit)
using JLD2
using Strided
Strided.enable_threads()

pkgversion(TensorKit) = v"0.12.3"


4

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = Rep[U₁×SU₂]((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
aspace = Rep[U₁×SU₂]((2, 0) => 2)

# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [2]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/Hubbard_t1.0U8.0mu4.0_ipepsEnv_D10chi200.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)
nothing

para[:χ] = 200


In [3]:
# 计算观测量
println("============== Calculating Obs ====================")
site1Obs = ["N"]                                # 计算这些单点观测量
site2Obs = ["hij", "SS", "NN", "Δₛ", "Δₛdag"]   # 计算这些两点观测量
# sites = [[x, y] for x in 1:Lx, y in 1:Ly]
Eg = 0.0
doping = 0.0
for xx in 1:Lx, yy in 1:Ly
    Obs1si = Cal_Obs_1site(ipeps, ipepsbar, envs, site1Obs, para; site=[xx, yy], get_op=get_op_Hubbard)
    @show Obs1si
    doping += get(Obs1si, "N", NaN)

    Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx + 1, yy], get_op=get_op_Hubbard)
    @show Obs2si
    Eg += get(Obs2si, "hij", NaN)
    Obs2si = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx, yy + 1], get_op=get_op_Hubbard)
    @show Obs2si
    Eg += get(Obs2si, "hij", NaN)

    # Obs2sidiag = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx+1, yy+1], get_op=get_op_Hubbard)
    # @show Obs2sidiag
    # Obs2sidiag = Cal_Obs_2site(ipeps, ipepsbar, envs, site2Obs, para; site1=[xx, yy], site2=[xx - 1, yy + 1], get_op=get_op_Hubbard)
end
GC.gc()
Eg = Eg / (Lx * Ly)
@show Eg
doping = doping / (Lx * Ly)
@show doping
nothing

============== Calculating Obs ====================
Fermionic! Calculating ["N"] at site [1, 1]
Obs1si = Dict("N" => 0.46363919848845064)
Fermionic! Calculating ["hij", "SS", "NN", "Δₛ", "Δₛdag"] at site [1, 1] and site [2, 1]
